# Tiền xử lý dữ liệu

In [133]:
#Importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [134]:
dataset = pd.read_csv('./Data/Data.csv')
dataset

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes
5,France,35.0,58000.0,Yes
6,Spain,NaN,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


In [135]:
# Tỉ lệ dữ liệu bị thiếu trong mỗi thuộc tính
dataset.isnull().mean() * 100

Country       0.0
Age          10.0
Salary       10.0
Purchased     0.0
dtype: float64

In [136]:
# Đếm số lượng
dataset.isnull().sum()

Country      0
Age          1
Salary       1
Purchased    0
dtype: int64

**Nhận xét:**
- Thuộc tính `Country`, `Purchased`(*target*) không có dữ liệu bị thiếu.
- Thuộc tính `Age`, `Salary` có dữ liệu bị thiếu.
- Tùy vào bài toán, xét các yếu tố như mức độ ảnh hưởng của thuộc tính đến target, ngưỡng dữ liệu thiếu để chọn cách thức xử lý các dữ liệu bị thiếu này:
    - Xóa các *sample* có dữ liệu bị thiếu nếu tỉ lệ dữ liệu thiếu dưới ngưỡng cho phép hoặc dữ liệu thiếu nằm trong thuộc tính ảnh hưởng ít đến target.
    - Chọn phương pháp điền giá trị vào các dữ liệu thiếu phù hợp trong các trường hợp khác.

Các thủ thuật tách đặc trưng - *X* và nhãn - *y*
|X|y|Trường hợp|
|-|-|-|
|`df.iloc[:,:-1]`|`df.iloc[:,-1]`|Khi biết nhãn nằm ở cột cuối|
|`df.loc[:,df,columns != 'target']`|`df.loc[:,'target]`|Không phụ thuộc vào vị trí cột|
|`df.drop(columns=['target'])`|`df['target']`|Có thể chọn các đặc trưng cần thiết cho `X`|



In [137]:
X = dataset.iloc[:,:-1] 
y = dataset.iloc[:,-1]

In [138]:
X

,Country,Age,Salary
0,France,44.0,72000.0
1,Spain,27.0,48000.0
2,Germany,30.0,54000.0
3,Spain,38.0,61000.0
4,Germany,40.0,NaN
5,France,35.0,58000.0
6,Spain,NaN,52000.0
7,France,48.0,79000.0
8,Germany,50.0,83000.0
9,France,37.0,67000.0


In [139]:
y

0     No
1    Yes
2     No
3     No
4    Yes
5    Yes
6     No
7    Yes
8     No
9    Yes
Name: Purchased, dtype: object

## Xử lý dữ liệu bị thiếu

Khai báo lớp `SimpleImputer`
```python
class SimpleImputer(
    *,
    missing_values: float / str / int / None = ...,
    strategy: str = "mean",
    fill_value: float / str / int / None = None,
    verbose: Int / str = "deprecated",
    copy: bool = True,
    add_indicator: bool = False,
    keep_empty_features: bool = False
)
```

Các tham số:
- `missing_values`: Xác định xem giá trị nào được xem là thiếu, mặc định là `np.nan`, dùng trong trường hợp có một số dữ liệu biểu diễn dữ liệu thiếu là *unknown*
- `strategy`: Xác định cách thay thế các giá trị bị thiếu.
    - `mean` - thay thế bằng trung bình của cột, áp dụng *numeric*, phổ biến trong dữ liệu liên tục, **Dễ ảnh hưởng bởi outliers**.
    - `median` - thay thế bằng trung vị cột, áp dụng *numeric*, giảm ảnh hưởng của ngoại lại, phù hợp với dữ liệu không đối xứng, **Không phản ánh đúng nếu dữ liệu ở dạng phân phối Gaussian** 
    - `most_frequent` - thay thế bằng giá trị xuất hiện nhiều nhất của cột, áp dụng *numeric & categorical*, giữ nguyên phân bố phổ biến của dữ liệu, **Không hiệu quả với dữ liệu phân bố đồng đều, mất tính đa dạng dữ liệu**
    - `constant` - thay thế bằng giá trị cố định, áp dụng *numeric & categorical*, đi với tham số `fill_value`, linh hoạt, giữ nguyên cấu trúc dữ liệu, **Không phản ánh đúng xu hướng dữ liệu thực tế**
- `copy=True`: tạo bản sao dữ liệu gốc.
- `add_indicator=True`: thêm một cột biểu thị vị trí của giá trị bị thiếu.
- `keep_empty_features=True`: những cột chứa toàn bộ giá trị bị thiếu sẽ được giữ lại thay vì bị loại bỏ

In [140]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
imputer.fit(X[['Age', 'Salary']].astype(float))
X[['Age', 'Salary']] = imputer.transform(X[['Age', 'Salary']])

In [141]:
X

,Country,Age,Salary
0,France,44.000000,72000.000000
1,Spain,27.000000,48000.000000
2,Germany,30.000000,54000.000000
3,Spain,38.000000,61000.000000
4,Germany,40.000000,63777.777778
5,France,35.000000,58000.000000
6,Spain,38.777778,52000.000000
7,France,48.000000,79000.000000
8,Germany,50.000000,83000.000000
9,France,37.000000,67000.000000


In [142]:
y

0     No
1    Yes
2     No
3     No
4    Yes
5    Yes
6     No
7    Yes
8     No
9    Yes
Name: Purchased, dtype: object

## Chuẩn hóa dữ liệu
Chuẩn hóa dữ liệu giúp đảm bảo các đặc trưng có thang đo đồng nhất.  
Một số thuật toán nhạy cảm với độ lớn của dữ liệu nên việc chuẩn hóa giúp tránh ảnh hưởng tiêu cực đến mô hình học máy.

### Các phương pháp chuẩn hóa
**Chuẩn hóa Min - Max**
- Đưa dữ liệu về một khoảng nhất định (thường là [0,1] hoặc [-1,1]).
- Phù hợp với dữ liệu không có hoặc đã xử lý các outliers.
$$
X' = \frac{X-X_{\text{min}}}{X_{\text{max}}-X_{\text{min}}}
$$
```python
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))  # Chuẩn hóa về khoảng [0,1]
```

**Chuẩn hóa Z-score**
- Chuyển dữ liệu về dạng có trung bình 0 và độ lệch chuẩn 1.
- Chuyển về phân phối chuẩn, không bị ảnh hưởng quá nhiều bởi outliers.
$$
X' = \frac{X-μ}{σ}
$$
```python
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
```

In [143]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=2024, stratify=y)

In [144]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
X_train[['Age', 'Salary']] = scaler.fit_transform(X_train[['Age', 'Salary']])
X_test[['Age', 'Salary']] = scaler.transform(X_test[['Age', 'Salary']])

In [145]:
X_train

,Country,Age,Salary
5,France,-0.304704,-0.278603
2,Germany,-1.077194,-0.670006
4,Germany,0.467785,0.286757
8,Germany,2.012764,2.167666
3,Spain,0.158790,0.014949
9,France,0.004292,0.602054
1,Spain,-1.540687,-1.257110
6,Spain,0.278955,-0.865707


## Xử lý dữ liệu Categorical
**Label Encoding**
- Chuyển danh mục thành số nguyên
- Sử dụng cho Ordinal (Danh mục có thứ tự)
```python
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
```

**One-hot Encoding**
- Biết mỗi giá trị danh mục thành cột riêng
- Sử dụng cho Nominal (Không có thứ tự)
```python
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
encoder = ColumnTransformer(transformers=[('cat', OneHotEncoder(), [0])], remainder='passthrough')
```

In [146]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer


#Ta có country là kiểu dữ liệu nominal
encoder = ColumnTransformer(transformers=[('cat', OneHotEncoder(), ['Country'])], remainder='passthrough')
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


In [147]:
#ColumnTransformer trả về numpy array
X_train = pd.DataFrame(X_train, columns=encoder.get_feature_names_out())
X_test = pd.DataFrame(X_test, columns=encoder.get_feature_names_out())

In [148]:
X_train

,cat__Country_France,cat__Country_Germany,cat__Country_Spain,remainder__Age,remainder__Salary
0,1.0,0.0,0.0,-0.304704,-0.278603
1,0.0,1.0,0.0,-1.077194,-0.670006
2,0.0,1.0,0.0,0.467785,0.286757
3,0.0,1.0,0.0,2.012764,2.167666
4,0.0,0.0,1.0,0.158790,0.014949
5,1.0,0.0,0.0,0.004292,0.602054
6,0.0,0.0,1.0,-1.540687,-1.257110
7,0.0,0.0,1.0,0.278955,-0.865707


In [149]:
y_train

array([1, 0, 1, 0, 0, 1, 1, 0])

Ngoài ra, khi biến phụ thuộc `y` muốn biến đổi theo ý muốn, có thể sử dụng map()
- Kiểm soát tốt giá trị ánh xạ, không cần `sklearn`
- Áp dụng linh hoạt với cả việc biến đổi các đặc trưng
```python
maping = {'No':0, 'Yes':1}
df['target'] = df['target'].map(mapping)
```

## Tổng kết
Data Preprocessing - Tiền xử lý dữ liệu: là bước quan trọng giúp dữ liệu sạch, nhất quán và phù hợp cho mô hình học máy
- Xử lý dữ liệu bị thiếu
- Mã hóa dữ liệu categorical
- Tách dữ liệu huấn luyện và kiểm tra
- Chuẩn hóa dữ liệu